In [4]:
import pandas as pd
import numpy as np
import os 
from datetime import datetime
import scipy as sp
import time
from datetime import datetime, date, timedelta
from dateutil.relativedelta import relativedelta
import io
from io import StringIO 
#import datedelta
import calendar
import  csv
import json
import random
import plotly
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.io as pio
from scipy import stats
def add_delta_cols(df):
    #df['delta_vector'] = 0
    #df['delta_max_vector'] = 0

    df['delta_vector'] = 0
    df['delta_max'] = 0
    df['upper_wick'] = 0
    df['lower_wick'] = 0
    
    dv_col = df.columns.get_loc('delta_vector')
    dm_col = df.columns.get_loc('delta_max')
    uw_col = df.columns.get_loc('upper_wick')
    lw_col = df.columns.get_loc('lower_wick')
    
    o_col = df.columns.get_loc('o')
    h_col = df.columns.get_loc('h')
    l_col = df.columns.get_loc('l')
    c_col = df.columns.get_loc('c')
    arr = df.values
    for i in range(arr.shape[0]):
        arr[i,dv_col] = arr[i,c_col] - arr[i,o_col]
        arr[i,dm_col] = arr[i,h_col] - arr[i,l_col]
        if arr[i,c_col] > arr[i,o_col]:
            arr[i,uw_col] = arr[i,h_col] - arr[i,c_col]
            arr[i,lw_col] = arr[i,o_col] - arr[i,l_col]
            
        elif arr[i,c_col] < arr[i,o_col]:
            arr[i,uw_col] = arr[i,h_col] - arr[i,o_col]
            arr[i,lw_col] = arr[i,c_col] - arr[i,l_col]   
            
    df = pd.DataFrame(arr,columns = df.columns)
    df['delta'] = abs(df['delta_vector'])
    
    return df
def load_df(pair = 'EUR_USD'
            ,granularity = 'M5'
            ,start = datetime(2016,1,1,0,0,0)
            ,end = datetime(2022,7,31,0,0,0)):
    dir_name = os.getcwd() + '/' + pair + '_' + granularity
    path = dir_name + '/' + str(date(start.year,start.month,start.day)) + '_' + str(date(end.year,end.month,end.day)) + '.csv'
    print('PATH:',path)
    df = pd.read_csv(path)
    df = add_delta_cols(df)
    print(df.shape)
    return df
def get_support(df,
                lookup_range = 1500,
                lookup_range2 = 2400,
                stop_range = 2200,
                candle_size = .001,
                bound_pips = .001
               ):
    print('GETTING SUPPORT INDICATOR')
    s = time.time()
    c_col = df.columns.get_loc('c')
    o_col = df.columns.get_loc('o')
    l_col = df.columns.get_loc('l')
    h_col = df.columns.get_loc('h')
    d_col = df.columns.get_loc('delta')
    df['support_lookup'] = 0
    df['support_indicator'] = 0

    lookup_col = df.columns.get_loc('support_lookup')
    new_col = df.columns.get_loc('support_indicator')

    reversal_col = df.columns.get_loc('c')   

    arr = df.values
    min_count = 0

    print('STOP RANGE:',stop_range)
    c = 0 

    for i in range(lookup_range,arr.shape[0]):
       # try:
        #DEFINE SUPPORT/RESISTANCE as the MIN or MAX of a lookup range
        min_ = min(arr[i - lookup_range : i + lookup_range2,reversal_col])

        # If the current val is at that reversal point:
        # Check and make sure we have an adequate movement off of that reversal point
        # this is defined by the pip_range and pip threshold, pip threshold is a user defined parameter
        if arr[i,reversal_col] == min_ and (i + lookup_range2 + stop_range < arr.shape[0]):
            lookup_ind = i
            val = arr[i,reversal_col]
            lookup_check = 0
            c = c + 1
            # Now that all of these conditions are met
            # We need to look into the future to see if we hit this reversal level, this is defined by the stop range
            for j in range(i + lookup_range2,i + lookup_range2 + stop_range):
                #If all of the following conditions are met, we will mark the array with a 1 signal to indicated we would place a trade                       
                #CANDLE BULL OR BEAR
                if arr[j,c_col] >= arr[j,o_col]:
                    bull = True
                else:
                    bull = False                        
                lower_bound = val - bound_pips
                upper_bound = val + bound_pips
                if i + lookup_range2 == j:
                    min_val = arr[j, reversal_col]
                else:
                    min_val = min(arr[i + lookup_range2 : j, reversal_col])
                    
                cur_val = arr[j - 1, reversal_col]

                if lookup_check == 0 and \
                bull == True and\
                arr[j,d_col] >= candle_size and\
                min_val >= lower_bound and \
                cur_val <= upper_bound and cur_val >= lower_bound:
                    arr[j,new_col] = 1
                    arr[j,lookup_col] = lookup_ind
                    lookup_check = 1
                    break

       # except:
        #    pass

    print('COUNT:',c)

    e = time.time()
    print('TOTAL FUNCTION TIME:',(e-s)/60,' MINUTES')
    df = pd.DataFrame(arr,columns = df.columns)
    print('SHAPE',df[df['support_indicator'] == 1].shape)
    return df

def get_resistance(df,
                lookup_range = 1500,
                lookup_range2 = 2400,
                stop_range = 2200,
                candle_size = .001,
                bound_pips = .001
               ):
    print('GETTING RESISTANCE INDICATOR')
    s = time.time()
    c_col = df.columns.get_loc('c')
    o_col = df.columns.get_loc('o')
    l_col = df.columns.get_loc('l')
    h_col = df.columns.get_loc('h')
    d_col = df.columns.get_loc('delta')
    df['resistance_lookup'] = 0
    df['resistance_indicator'] = 0

    lookup_col = df.columns.get_loc('resistance_lookup')
    new_col = df.columns.get_loc('resistance_indicator')

    reversal_col = df.columns.get_loc('c')   

    arr = df.values
    min_count = 0

    print('STOP RANGE:',stop_range)
    c = 0 

    for i in range(lookup_range,arr.shape[0]):
       # try:
        #DEFINE SUPPORT/RESISTANCE as the MIN or MAX of a lookup range
        max_ = max(arr[i - lookup_range : i + lookup_range2,reversal_col])

        # If the current val is at that reversal point:
        # Check and make sure we have an adequate movement off of that reversal point
        # this is defined by the pip_range and pip threshold, pip threshold is a user defined parameter
        if arr[i,reversal_col] == max_ and (i + lookup_range2 + stop_range < arr.shape[0]):
            lookup_ind = i
            val = arr[i,reversal_col]
            lookup_check = 0
            c = c + 1
            # Now that all of these conditions are met
            # We need to look into the future to see if we hit this reversal level, this is defined by the stop range
            for j in range(i + lookup_range2,i + lookup_range2 + stop_range):
                #If all of the following conditions are met, we will mark the array with a 1 signal to indicated we would place a trade                       
                #CANDLE BULL OR BEAR
                if arr[j,c_col] >= arr[j,o_col]:
                    bull = True
                else:
                    bull = False                        
                lower_bound = val - bound_pips
                upper_bound = val + bound_pips
                if i + lookup_range2 == j:
                    max_val = arr[j, reversal_col]
                else:
                    max_val = max(arr[i + lookup_range2 : j, reversal_col])
                cur_val = arr[j - 1, reversal_col]

                if lookup_check == 0 and \
                bull == False and\
                arr[j,d_col] >= candle_size and\
                max_val <= upper_bound and \
                cur_val <= upper_bound and cur_val >= lower_bound:
                    arr[j,new_col] = 1
                    arr[j,lookup_col] = lookup_ind
                    lookup_check = 1
                    break

       # except:
        #    pass

    print('COUNT:',c)

    e = time.time()
    print('TOTAL FUNCTION TIME:',(e-s)/60,' MINUTES')
    df = pd.DataFrame(arr,columns = df.columns)
    print('SHAPE',df[df['resistance_indicator'] == 1].shape)
    return df





def run_single_pair_sim(pair,
                lookup_range,
                lookup_range2,
                stop_range,
                candle_size,
                bound_pips,
                sls,
                tps,
                spread):
    
    def strategy_tester_buy(df,col = 'support_indicator',sl = .1,tp = .5,num_trades = 300,risk_size = .01,entry = 'c',pr = True,spread = .01):
        def simulate_results(
            num_trades = 200,
            acc_val = 10000,
            rr = 6,
            risk_size = .02,
            win_rate = .2,
            pr = True):

            start = acc_val
            for i in range(num_trades):
                rand = random.random()
                if rand > win_rate:
                    acc_val = acc_val - (acc_val * risk_size)
                else:
                    acc_val = acc_val + (acc_val * (risk_size * rr))
            if pr:
                print('STARTING VAL:',round(start),' ENDING VAL:',round(acc_val))
            return round(acc_val)

        col_num = df.columns.get_loc(col)
        h_col = df.columns.get_loc('h')
        l_col = df.columns.get_loc('l')
        c_col = df.columns.get_loc('c')
        entry_col = df.columns.get_loc(entry)
        arr = df.values
        temp_df = df[df[col] == 1]

        trade_list = []
        trade_res = []
        wins = []
        losses = []
        for i in range(arr.shape[0]):

            if arr[i,col_num] == 1:
                val = arr[i,entry_col]
                trade_id = 0
                try:
                    for j in range(i + 1,i + 10000):
                        if i not in trade_list:
                            if trade_id == 0:
                                if arr[j,l_col] <= val - sl + spread:
                                    trade_res.append(-10)
                                    trade_list.append(i)
                                    losses.append(i)
                                    trade_id = 1

                                    break
                                if arr[j,h_col] >= val + tp:
                                    trade_res.append(50)
                                    trade_list.append(i)
                                    wins.append(i)
                                    trade_id = 1
                                    break
                except:
                    pass
        win_rate = len(wins) / len(trade_list)
        rr = (tp - spread) / (sl + spread)
       # num_trades = len(trade_list)
      #  print('WIN RATE',win_rate,' RR:',rr, 'NUM TRADES:',num_trades,' RISK SIZE:',risk_size)
        vals = []


        acc_val = 10000



        for i in range(30):

            final_val = simulate_results(num_trades = num_trades,acc_val = acc_val,rr = rr,risk_size = risk_size,win_rate = win_rate,pr = pr)
            vals.append(final_val)

        return np.median(vals),win_rate,wins,losses

    def strategy_tester_sell(df,col = 'support_indicator',sl = .1,tp = .5,num_trades = 300,risk_size = .01,entry = 'c',pr = True,spread = .01):
        def simulate_results(
            num_trades = 200,
            acc_val = 10000,
            rr = 6,
            risk_size = .02,
            win_rate = .2,
            pr = True):

            start = acc_val
            for i in range(num_trades):
                rand = random.random()
                if rand > win_rate:
                    acc_val = acc_val - (acc_val * risk_size)
                else:
                    acc_val = acc_val + (acc_val * (risk_size * rr))
            if pr:
                print('STARTING VAL:',round(start),' ENDING VAL:',round(acc_val))
            return round(acc_val)

        col_num = df.columns.get_loc(col)
        h_col = df.columns.get_loc('h')
        l_col = df.columns.get_loc('l')
        c_col = df.columns.get_loc('c')
        entry_col = df.columns.get_loc(entry)
        arr = df.values
        temp_df = df[df[col] == 1]

        trade_list = []
        trade_res = []
        wins = []
        losses = []
        for i in range(arr.shape[0]):

            if arr[i,col_num] == 1:
                val = arr[i,entry_col]
                try:
                    for j in range(i + 1,i + 10000):
                        if i not in trade_list:
                            if arr[j,h_col] >= val + sl - spread:
                                trade_res.append(-10)
                                trade_list.append(i)
                                losses.append(i)
                                break
                            if arr[j,l_col] <= val - tp:
                                trade_res.append(50)
                                trade_list.append(i)
                                wins.append(i)
                                break
                except:
                    pass
        win_rate = len(wins) / len(trade_list)
        rr = (tp - spread) / (sl + spread)
       # num_trades = len(trade_list)
      #  print('WIN RATE',win_rate,' RR:',rr, 'NUM TRADES:',num_trades,' RISK SIZE:',risk_size)
        vals = []


        acc_val = 10000



        for i in range(30):

            final_val = simulate_results(num_trades = num_trades,acc_val = acc_val,rr = rr,risk_size = risk_size,win_rate = win_rate,pr = pr)
            vals.append(final_val)

        return np.median(vals),win_rate,wins,losses


    s = time.time()
    df = load_df(pair = pair,granularity ='M5',start = datetime(2016,1,1,0,0,0),end = datetime(2022,7,31,0,0,0))
    print(df.shape)
    df_ = df.copy()

    print('RUNNING SINGLE PARAMETER LOOP FOR PARAMS:',
          'INSTRUMENT:',pair,
          'lookup_range',lookup_range,
          'lookup_range2', lookup_range2,
                    'stop_range', stop_range,
                    'candle_size' , candle_size,
                    'bound_pips' , bound_pips)

    df = get_support(df,
                    lookup_range = lookup_range,
                    lookup_range2 = lookup_range2,
                    stop_range = stop_range,
                    candle_size = candle_size,
                    bound_pips = bound_pips
                    )

    df_ = get_resistance(df_,
                    lookup_range = lookup_range,
                    lookup_range2 = lookup_range2,
                    stop_range = stop_range,
                    candle_size = candle_size,
                    bound_pips = bound_pips
                   )


    cols = ['instrument',
           'buy_support_val',
           'sell_resistance_val',
           'buy_resistance_val',
           'sell_support_val',
            'max_ending_val',
            'sl',
            'tp',
            'RR',
           'win_rate1',
           'win_rate2',
           'win_rate3',
           'win_rate4',
            'lookup_range',
            'lookup_range2',
            'stop_range',
            'candle_size',
            'bound_pips',
            'support_number_of_trades',
            'resistance_number_of_trades'
          ]

    trade_strategy = 'support_indicator'
    trade_strategy2 = 'resistance_indicator'

    #trade_strategy = 'resistance_indicatorc'
    num_trades = 400
    risk_size = .01

    starting_val = 10000
    instrument = pair
    timeframe = 'M5'
    temp_lst = []
    actual_number_of_trades = df[df[trade_strategy] == 1].shape[0]
    actual_number_of_trades2 = df_[df_[trade_strategy2] == 1].shape[0]


    for sl in sls:
        for tp in tps:
            if actual_number_of_trades == 0:
                ending_val,win_rate,wins,losses = 0,0,0,0
                ending_val4,win_rate4,wins4,losses4 = 0,0,0,0
            else:
                ending_val,win_rate,wins,losses = strategy_tester_buy(df,col = trade_strategy,sl = sl,tp = tp,num_trades = num_trades,risk_size = risk_size,entry = 'c',pr = False,spread = spread)
                ending_val4,win_rate4,wins4,losses4 = strategy_tester_sell(df,col = trade_strategy,sl = sl,tp = tp,num_trades = num_trades,risk_size = risk_size,entry = 'c',pr = False,spread = spread)   
                
            if actual_number_of_trades2 == 0:
                ending_val2,win_rate2,wins2,losses2 = 0,0,0,0
                ending_val3,win_rate3,wins3,losses3 = 0,0,0,0
            else:
                ending_val2,win_rate2,wins2,losses2 = strategy_tester_sell(df_,col = trade_strategy2,sl = sl,tp = tp,num_trades = num_trades,risk_size = risk_size,entry = 'c',pr = False,spread = spread)
                ending_val3,win_rate3,wins3,losses3 = strategy_tester_buy(df_,col = trade_strategy2,sl = sl,tp = tp,num_trades = num_trades,risk_size = risk_size,entry = 'c',pr = False,spread = spread) 
                
                
            max_ending_val = max(ending_val,
                   ending_val2,
                   ending_val3,
                   ending_val4)
            lst = [instrument,
                   ending_val,
                   ending_val2,
                   ending_val3,
                   ending_val4,
                   max_ending_val,
                    sl,
                    tp,
                    round(tp / sl,4),
                   round(win_rate,4),
                   round(win_rate2,4),
                   round(win_rate3,4),
                   round(win_rate4,4),
                    lookup_range,
                    lookup_range2,
                    stop_range,
                    candle_size,
                    bound_pips,
                   actual_number_of_trades,
                   actual_number_of_trades2
                  ]
            temp_lst.append(lst)

    final_df = pd.DataFrame(temp_lst,columns = cols)
    final_df = final_df.sort_values(by = 'max_ending_val',ascending = False)
    
    e = time.time()
    best_buy_support = final_df['buy_support_val'].max()
    best_sell_resistance = final_df['sell_resistance_val'].max()
    best_buy_resistance = final_df['buy_resistance_val'].max()
    best_sell_support = final_df['sell_support_val'].max()
    max_val_all = final_df['max_ending_val'].max()
    print('MAX VAL:',round(max_val_all),'||best buy support:',round(best_buy_support),'||best sell resistance:',
          round(best_sell_resistance),'||best buy resistance:',round(best_buy_resistance),'||best sell support:',round(best_sell_support))


    print('TOTAL SINGLE PAIR SIM TIME:',round((e-s)/60,1),'MINUTES')
    return final_df

def run_all_pairs_sim_loop(
                    lookup_range,
                    lookup_range2,
                    stop_range,
                    candle_size ,
                    bound_pips,
                    spread):
    dir_name = os.getcwd() + '/reversal_counter_sim_results/' + str(lookup_range) + '_' + str(lookup_range2) + '_' + str(stop_range) + '_' + str(candle_size) + '_' + str(bound_pips)
    print(dir_name)
    isdir = os.path.isdir(dir_name) 
    if isdir:
        pass
    else:
        print('MAKING DIRECTORY:',dir_name)
        os.mkdir(dir_name) 

    if len(os.listdir(dir_name)) > 0:
        print('file already exists in this directory, passing...')
    else:
        print('file does not exist, running sim')


        all_pairs=['GBP_JPY','EUR_USD','USD_JPY','EUR_JPY','AUD_JPY','CAD_JPY','CHF_JPY',
                   'GBP_USD','USD_CAD','USD_CHF','AUD_USD',
                   'NZD_USD','AUD_CAD','AUD_CHF','AUD_NZD','CAD_CHF',
                   'EUR_AUD','EUR_CAD','EUR_CHF','NZD_CAD','NZD_CHF'] 


        final_df = pd.DataFrame([0])    
        for pair in all_pairs:
            print(pair)  

            if 'JPY' in pair:
                sls = [.05,.075,.1,.125,.15,.2,.25]
                tps = [.1,.15,.2,.25,.3,.4,.5,.7,.9,1.1,1.3,1.5]
                spread = .01
                candle_size_ = candle_size * 100
                bound_pips_ = bound_pips * 100
            else:
                sls = [.0005,.00075,.001,.00125,.0015,.002,.0025]
                tps = [.001,.0015,.002,.0025,.003,.004,.005,.007,.009,.011,.013,.015]
                spread = .0001
                candle_size_ = candle_size 
                bound_pips_ = bound_pips        

            df = run_single_pair_sim(pair,
                            lookup_range,
                            lookup_range2,
                            stop_range,
                            candle_size_,
                            bound_pips_,
                            sls,
                            tps,
                            spread)    
            if final_df.shape[0] == 1:
                final_df = df
            else:
                final_df = pd.concat([final_df,df],axis = 0)

        final_df = final_df.sort_values(by = 'max_ending_val',ascending = False)    

        gb = final_df.groupby('instrument').agg({'max_ending_val':max,'buy_support_val':max,'sell_resistance_val':max,'buy_resistance_val':max,'sell_support_val':max})
        gb = gb.reset_index()
        gb = gb.sort_values(by = 'max_ending_val',ascending = False)    
        for i in range(gb.shape[0]):
            print('pair',gb['instrument'].iloc[i],'||MAX VAL',gb['max_ending_val'].iloc[i],'||best buy support:',gb['buy_support_val'].iloc[i],'||best sell resistance:',
                  gb['sell_resistance_val'].iloc[i],'||best buy resistance:',gb['buy_resistance_val'].iloc[i],'||best sell support:',gb['sell_support_val'].iloc[i])

        print('MEDIANS:',np.median(gb['max_ending_val']),np.median(gb['buy_support_val']),np.median(gb['sell_resistance_val']),np.median(gb['buy_resistance_val']),np.median(gb['sell_support_val']))

        path = dir_name + '/' + str(round(np.median(gb['max_ending_val']))) + '_' + str(round(np.median(gb['buy_support_val']))) + '_' + str(round(np.median(gb['sell_resistance_val']))) + '_' + str(round(np.median(gb['buy_resistance_val']))) + '_' + str(round(np.median(gb['sell_support_val'])))
        path = path + '.csv'
        print('FINAL PATH:',path)
        print('SAVING DATA')
        final_df.to_csv(path,index = False)
        print()
        print()
        print()
        print()
        print()
        
def get_support(df,
                lookup_range = 1500,
                pip_delta = .001,
                time_range = 11,
                high_spread = .0005,
                low_spread = 0
               ):
    print('GETTING SUPPORT INDICATOR')
    print('LOOKUP RANGE AND PIP DELTA:',lookup_range,pip_delta)
    s = time.time()
    c_col = df.columns.get_loc('c')
    o_col = df.columns.get_loc('o')
    l_col = df.columns.get_loc('l')
    h_col = df.columns.get_loc('h')
    d_col = df.columns.get_loc('delta')
    minute_col = df.columns.get_loc('minute_col')
    df['support_indicator'] = 0
    new_col = df.columns.get_loc('support_indicator')
    arr = df.values
    min_count = 0
    c = 0 
   # high_spread = .0003
    #low_spread = .0001
    #low_spread = 0
    #high_spread = .0005
    
    for i in range(max(lookup_range,time_range + 1),arr.shape[0]):
       # try:
        #DEFINE SUPPORT/RESISTANCE as the MIN or MAX of a lookup range

        if (arr[i - time_range,o_col] - arr[i,c_col]) >= pip_delta and arr[i,minute_col] == 55:
            arr[i,new_col] = 1

    print('COUNT:',c)

    e = time.time()
    print('TOTAL FUNCTION TIME:',(e-s)/60,' MINUTES')
    df = pd.DataFrame(arr,columns = df.columns)
    print('SHAPE',df[df['support_indicator'] == 1].shape)
    return df

def load_df(pair = 'EUR_USD'
            ,granularity = 'M5'
            ,start = datetime(2016,1,1,0,0,0)
            ,end = datetime(2022,7,31,0,0,0)):
    def convert_timestamp(df):
        time_col = df.columns.get_loc('time')
        df['hour_col'] = 0
        df['xau_morning'] = 0
        new_col = df.columns.get_loc('hour_col')
        new_col2 = df.columns.get_loc('xau_morning')
        arr = df.values 
        for i in range(arr.shape[0]):
            arr[i,new_col] = pd.Timestamp(arr[i,time_col]).hour
            if arr[i,new_col] == 11:
                arr[i,new_col2] = 1
                
        return pd.DataFrame(arr,columns = df.columns)
    dir_name = os.getcwd() + '/' + pair + '_' + granularity
    path = dir_name + '/' + str(date(start.year,start.month,start.day)) + '_' + str(date(end.year,end.month,end.day)) + '.csv'
    print('PATH:',path)
    df = pd.read_csv(path)
    df = add_delta_cols(df)
    df = convert_timestamp(df)
    print(df.shape)
    return df
pair = 'XAU_USD'
df = load_df(pair = pair,granularity ='H1',start = datetime(2016,1,1,0,0,0),end = datetime(2022,7,31,0,0,0))
print(df.shape)
df.head()



PATH: /Users/aidanmcconnell/Documents/MARKET_ANALYTICS/forex/Macro/XAU_USD_H1/2016-01-01_2022-07-31.csv
(39094, 15)
(39094, 15)


,time,volume,o,h,l,c,utc_timestamp,est_timestamp,delta_vector,delta_max,upper_wick,lower_wick,delta,hour_col,xau_morning
0,2016-01-01T13:00:00.000000000Z,2,1060.588,1060.588,1060.588,1060.588,2016-01-01 13:00:00+00:00,2016-01-01 08:00:00-05:00,0.0,0.0,0,0,0.0,13,0
1,2016-01-01T14:00:00.000000000Z,1,1060.588,1060.588,1060.588,1060.588,2016-01-01 14:00:00+00:00,2016-01-01 09:00:00-05:00,0.0,0.0,0,0,0.0,14,0
2,2016-01-01T15:00:00.000000000Z,1,1060.588,1060.588,1060.588,1060.588,2016-01-01 15:00:00+00:00,2016-01-01 10:00:00-05:00,0.0,0.0,0,0,0.0,15,0
3,2016-01-03T23:00:00.000000000Z,3977,1062.7,1066.887,1061.891,1063.238,2016-01-03 23:00:00+00:00,2016-01-03 18:00:00-05:00,0.538,4.996,3.649,0.809,0.538,23,0
4,2016-01-04T00:00:00.000000000Z,3792,1063.354,1064.986,1062.84,1064.727,2016-01-04 00:00:00+00:00,2016-01-03 19:00:00-05:00,1.373,2.146,0.259,0.514,1.373,0,0


In [38]:
def get_support(df):
    c = df.columns.get_loc('c')
    o = df.columns.get_loc('o')
    l = df.columns.get_loc('l')
    df['support_indicator'] = 0
    new_col = df.columns.get_loc('support_indicator')
    d = df.columns.get_loc('delta')
    
    arr = df.values
    for i in range(arr.shape[0]):
        if arr[i,c] == arr[i,l] and arr[i,c] < arr[i,o] and arr[i,d] > 0 and arr[i,c] <= arr[i - 100,c] - 25:
            arr[i + 1,new_col] = 1
    df = pd.DataFrame(arr,columns = df.columns)
    print('SHAPE',df[df['support_indicator'] == 1].shape)
    return df
df = get_support(df)
df.head()

SHAPE (39, 16)


,time,volume,o,h,l,c,utc_timestamp,est_timestamp,delta_vector,delta_max,upper_wick,lower_wick,delta,hour_col,xau_morning,support_indicator
0,2016-01-01T13:00:00.000000000Z,2,1060.588,1060.588,1060.588,1060.588,2016-01-01 13:00:00+00:00,2016-01-01 08:00:00-05:00,0.0,0.0,0,0,0.0,13,0,0
1,2016-01-01T14:00:00.000000000Z,1,1060.588,1060.588,1060.588,1060.588,2016-01-01 14:00:00+00:00,2016-01-01 09:00:00-05:00,0.0,0.0,0,0,0.0,14,0,0
2,2016-01-01T15:00:00.000000000Z,1,1060.588,1060.588,1060.588,1060.588,2016-01-01 15:00:00+00:00,2016-01-01 10:00:00-05:00,0.0,0.0,0,0,0.0,15,0,0
3,2016-01-03T23:00:00.000000000Z,3977,1062.7,1066.887,1061.891,1063.238,2016-01-03 23:00:00+00:00,2016-01-03 18:00:00-05:00,0.538,4.996,3.649,0.809,0.538,23,0,0
4,2016-01-04T00:00:00.000000000Z,3792,1063.354,1064.986,1062.84,1064.727,2016-01-04 00:00:00+00:00,2016-01-03 19:00:00-05:00,1.373,2.146,0.259,0.514,1.373,0,0,0


In [39]:
df[df['support_indicator'] == 1]['delta_vector'].mean()

-0.5734102564102462

In [40]:
df[df['support_indicator'] == 1]['upper_wick'].mean()

0.9807179487179499

In [41]:
df[(df['support_indicator'] == 1) & (df['delta_vector'] < 0)]['upper_wick'].mean()

0.9992380952380808

In [42]:
def get_support(df):
    c = df.columns.get_loc('c')
    o = df.columns.get_loc('o')
    l = df.columns.get_loc('l')
    df['support_indicator'] = 0
    new_col = df.columns.get_loc('support_indicator')
    d = df.columns.get_loc('delta')
    
    arr = df.values
    for i in range(arr.shape[0]):
        if arr[i,c] == arr[i,l] and arr[i,c] < arr[i,o] and arr[i,d] > 0:
            arr[i + 1,new_col] = 1
    df = pd.DataFrame(arr,columns = df.columns)
    print('SHAPE',df[df['support_indicator'] == 1].shape)
    return df
df = get_support(df)
df.head()

SHAPE (284, 16)


,time,volume,o,h,l,c,utc_timestamp,est_timestamp,delta_vector,delta_max,upper_wick,lower_wick,delta,hour_col,xau_morning,support_indicator
0,2016-01-01T13:00:00.000000000Z,2,1060.588,1060.588,1060.588,1060.588,2016-01-01 13:00:00+00:00,2016-01-01 08:00:00-05:00,0.0,0.0,0,0,0.0,13,0,0
1,2016-01-01T14:00:00.000000000Z,1,1060.588,1060.588,1060.588,1060.588,2016-01-01 14:00:00+00:00,2016-01-01 09:00:00-05:00,0.0,0.0,0,0,0.0,14,0,0
2,2016-01-01T15:00:00.000000000Z,1,1060.588,1060.588,1060.588,1060.588,2016-01-01 15:00:00+00:00,2016-01-01 10:00:00-05:00,0.0,0.0,0,0,0.0,15,0,0
3,2016-01-03T23:00:00.000000000Z,3977,1062.7,1066.887,1061.891,1063.238,2016-01-03 23:00:00+00:00,2016-01-03 18:00:00-05:00,0.538,4.996,3.649,0.809,0.538,23,0,0
4,2016-01-04T00:00:00.000000000Z,3792,1063.354,1064.986,1062.84,1064.727,2016-01-04 00:00:00+00:00,2016-01-03 19:00:00-05:00,1.373,2.146,0.259,0.514,1.373,0,0,0


In [43]:
df[df['support_indicator'] == 1]['delta_vector'].mean()

-0.11549647887323919

In [44]:
df[(df['support_indicator'] == 1) & (df['delta_vector'] < 0)]['upper_wick'].mean()

0.7630571428571427

In [29]:
df[df['support_indicator'] == 1]['delta_vector'].mean()

-0.0887812499999967

In [27]:
df[df['support_indicator'] == 1]['delta_vector'].mean()

-0.39017647058823757

In [17]:
df[df['support_indicator'] == 1]['delta_vector'].mean()

-0.11549647887323919

In [16]:
df['delta'].mean()

1.7627330025067762

In [19]:
24 * 4

96

In [13]:
df[df['support_indicator'] == 1].shape

(284, 16)

In [8]:
df[df['c'] == df['h']].shape

(326, 15)

In [60]:
def get_support(df):
    c = df.columns.get_loc('c')
    o = df.columns.get_loc('o')
    l = df.columns.get_loc('l')
    df['support_indicator'] = 0
    new_col = df.columns.get_loc('support_indicator')
    d = df.columns.get_loc('delta')
    
    arr = df.values
    for i in range(arr.shape[0]):
        if arr[i,c] == arr[i,l] and arr[i,o] - arr[i,c] >= .001 and arr[i,d] > 0 and arr[i,c] <= arr[i - 100,c] - .005:
            arr[i + 1,new_col] = 1
    df = pd.DataFrame(arr,columns = df.columns)
    print('SHAPE',df[df['support_indicator'] == 1].shape)
    return df

pair = 'EUR_USD'
df = load_df(pair = pair,granularity ='H1',start = datetime(2016,1,1,0,0,0),end = datetime(2022,7,31,0,0,0))
df = get_support(df)
df.head()

PATH: /Users/aidanmcconnell/Documents/MARKET_ANALYTICS/forex/Macro/EUR_USD_H1/2016-01-01_2022-07-31.csv
(41152, 15)
SHAPE (48, 16)


,time,volume,o,h,l,c,utc_timestamp,est_timestamp,delta_vector,delta_max,upper_wick,lower_wick,delta,hour_col,xau_morning,support_indicator
0,2016-01-03T22:00:00.000000000Z,289,1.08743,1.08746,1.08667,1.0867,2016-01-03 22:00:00+00:00,2016-01-03 17:00:00-05:00,-0.00073,0.00079,0.00003,0.00003,0.00073,22,0,0
1,2016-01-03T23:00:00.000000000Z,911,1.08665,1.0867,1.08454,1.08519,2016-01-03 23:00:00+00:00,2016-01-03 18:00:00-05:00,-0.00146,0.00216,0.00005,0.00065,0.00146,23,0,0
2,2016-01-04T00:00:00.000000000Z,794,1.08524,1.08578,1.08432,1.08454,2016-01-04 00:00:00+00:00,2016-01-03 19:00:00-05:00,-0.0007,0.00146,0.00054,0.00022,0.0007,0,0,0
3,2016-01-04T01:00:00.000000000Z,1565,1.08456,1.08617,1.08271,1.08597,2016-01-04 01:00:00+00:00,2016-01-03 20:00:00-05:00,0.00141,0.00346,0.0002,0.00185,0.00141,1,0,0
4,2016-01-04T02:00:00.000000000Z,1728,1.086,1.08757,1.08586,1.08716,2016-01-04 02:00:00+00:00,2016-01-03 21:00:00-05:00,0.00116,0.00171,0.00041,0.00014,0.00116,2,0,0


In [51]:
df[df['support_indicator'] == 1]['delta_vector'].mean()

5.923076923076929e-05

In [47]:
df[(df['support_indicator'] == 1) & (df['delta_vector'] < 0)]['upper_wick'].mean()


0.00027522727272727676

In [52]:
df[(df['support_indicator'] == 1)]['upper_wick'].mean()

0.00030494505494505674

In [53]:
(df[(df['support_indicator'] == 1)]['upper_wick'].mean()) * 10000

3.0494505494505675

In [54]:
(df[df['support_indicator'] == 1]['delta_vector'].mean()) * 10000

0.5923076923076929

In [57]:
(df[df['support_indicator'] == 1]['delta_vector'].mean()) * 10000

-0.40689655172404715

In [61]:
(df[df['support_indicator'] == 1]['delta_vector'].mean()) * 10000

2.9249999999999647

In [62]:
(df[(df['support_indicator'] == 1)]['lower_wick'].mean()) * 10000

3.564583333333417

In [64]:
(df[(df['support_indicator'] == 1) & (df['delta_vector'] > 0)]['lower_wick'].mean()) * 10000

3.2312500000001436

In [65]:
wins = df[(df['support_indicator'] == 1) & (df['delta_vector'] > 0)]
wins = wins.sort_values(by = 'delta_vector',ascending = False)
wins.shape

(32, 16)

In [66]:
(df[(df['support_indicator'] == 1) & (df['delta_vector'] > 0)]['delta_vector'].mean()) * 10000

8.212500000000025

In [71]:
df[(df['support_indicator'] == 1) & (df['delta_vector'] > 0) & (df['lower_wick'] >= .0001)].shape

(20, 16)

In [91]:
def get_support(df):
    c = df.columns.get_loc('c')
    o = df.columns.get_loc('o')
    l = df.columns.get_loc('l')
    df['support_indicator'] = 0
    new_col = df.columns.get_loc('support_indicator')
    d = df.columns.get_loc('delta')
    
    arr = df.values
    for i in range(arr.shape[0]):
        if arr[i,c] == arr[i,l] and arr[i,o] - arr[i,c] >= .0007 and arr[i,d] > 0 and arr[i,c] <= arr[i - 150,c] - .008:
            arr[i + 1,new_col] = 1
    df = pd.DataFrame(arr,columns = df.columns)
    print('SHAPE',df[df['support_indicator'] == 1].shape)
    return df

pair = 'EUR_USD'
df = load_df(pair = pair,granularity ='H1',start = datetime(2016,1,1,0,0,0),end = datetime(2022,7,31,0,0,0))
df = get_support(df)
df.head()

PATH: /Users/aidanmcconnell/Documents/MARKET_ANALYTICS/forex/Macro/EUR_USD_H1/2016-01-01_2022-07-31.csv
(41152, 15)
SHAPE (70, 16)


,time,volume,o,h,l,c,utc_timestamp,est_timestamp,delta_vector,delta_max,upper_wick,lower_wick,delta,hour_col,xau_morning,support_indicator
0,2016-01-03T22:00:00.000000000Z,289,1.08743,1.08746,1.08667,1.0867,2016-01-03 22:00:00+00:00,2016-01-03 17:00:00-05:00,-0.00073,0.00079,0.00003,0.00003,0.00073,22,0,0
1,2016-01-03T23:00:00.000000000Z,911,1.08665,1.0867,1.08454,1.08519,2016-01-03 23:00:00+00:00,2016-01-03 18:00:00-05:00,-0.00146,0.00216,0.00005,0.00065,0.00146,23,0,0
2,2016-01-04T00:00:00.000000000Z,794,1.08524,1.08578,1.08432,1.08454,2016-01-04 00:00:00+00:00,2016-01-03 19:00:00-05:00,-0.0007,0.00146,0.00054,0.00022,0.0007,0,0,0
3,2016-01-04T01:00:00.000000000Z,1565,1.08456,1.08617,1.08271,1.08597,2016-01-04 01:00:00+00:00,2016-01-03 20:00:00-05:00,0.00141,0.00346,0.0002,0.00185,0.00141,1,0,0
4,2016-01-04T02:00:00.000000000Z,1728,1.086,1.08757,1.08586,1.08716,2016-01-04 02:00:00+00:00,2016-01-03 21:00:00-05:00,0.00116,0.00171,0.00041,0.00014,0.00116,2,0,0


In [107]:
def test_future(df,pair,column = 'support_indicator'):
    if 'JPY' in pair:
        mult = 100
    else:
        mult = 100 * 100
    c_col = df.columns.get_loc('c')
    col = df.columns.get_loc(column)
    
    lst = []
    arr = df.values
    for i in range(arr.shape[0] - 101):
        if arr[i,col] == 1:
            c = arr[i,c_col]

            lst.append([mult * (arr[i + 1,c_col] - c),
                        mult * (arr[i + 2,c_col] - c),
                        mult * (arr[i + 3,c_col] - c),
                        mult * (arr[i + 4,c_col] - c), 
                        mult * (arr[i + 5,c_col] - c), 
                        mult * (arr[i + 6,c_col] - c)
                       ])
    df2 = pd.DataFrame(lst,columns = ['next_hour','next_two_hours','next_three_hours','next_four_hours','next_five_hours','next_six_hours'])  
    print('MEDIAN:')
    print(df2.median())
    print()
    print('MEAN:')
    print(df2.mean())
    print()
    print('SUM:')
    print(print(pd.DataFrame(df2.sum()).T))
    
    return pd.DataFrame(df2.median(),columns = [pair]).T,pd.DataFrame(df2.mean(),columns = [pair]).T,pd.DataFrame(df2.sum(),columns = [pair]).T
def test_future2(df,pair,column = 'support_indicator'):
    if 'JPY' in pair:
        mult = 100
    else:
        mult = 100 * 100
    c_col = df.columns.get_loc('c')
    col = df.columns.get_loc(column)
    dv_col = df.columns.get_loc('delta_vector')
    o_col = df.columns.get_loc('o')
    
    lst = []
    arr = df.values
    for i in range(arr.shape[0] - 101):
        if arr[i,col] == 1 and arr[i,dv_col] < 0:
            c = arr[i,o_col]

            lst.append([mult * (arr[i ,c_col] - c),
                mult * (arr[i + 1,c_col] - c),
                        mult * (arr[i + 2,c_col] - c),
                        mult * (arr[i + 3,c_col] - c),
                        mult * (arr[i + 4,c_col] - c), 
                        mult * (arr[i + 5,c_col] - c), 
                        mult * (arr[i + 6,c_col] - c)
                       ])
    df2 = pd.DataFrame(lst,columns = ['current_hour','next_hour','next_two_hours','next_three_hours','next_four_hours','next_five_hours','next_six_hours'])  
    print('MEDIAN:')
    print(df2.median())
    print()
    print('MEAN:')
    print(df2.mean())
    print()
    print('SUM:')
    print(print(pd.DataFrame(df2.sum()).T))
    
    return pd.DataFrame(df2.median(),columns = [pair]).T,pd.DataFrame(df2.mean(),columns = [pair]).T,pd.DataFrame(df2.sum(),columns = [pair]).T

def test_future3(df,pair,column = 'support_indicator'):
    if 'JPY' in pair:
        mult = 100
    else:
        mult = 100 * 100
    c_col = df.columns.get_loc('c')
    col = df.columns.get_loc(column)
    dv_col = df.columns.get_loc('delta_vector')
    o_col = df.columns.get_loc('o')
    
    lst = []
    arr = df.values
    for i in range(arr.shape[0] - 101):
        if arr[i,col] == 1 and arr[i,dv_col] > 0:
            c = arr[i,o_col]

            lst.append([mult * (arr[i ,c_col] - c),
                mult * (arr[i + 1,c_col] - c),
                        mult * (arr[i + 2,c_col] - c),
                        mult * (arr[i + 3,c_col] - c),
                        mult * (arr[i + 4,c_col] - c), 
                        mult * (arr[i + 5,c_col] - c), 
                        mult * (arr[i + 6,c_col] - c)
                       ])
    df2 = pd.DataFrame(lst,columns = ['current_hour','next_hour','next_two_hours','next_three_hours','next_four_hours','next_five_hours','next_six_hours'])  
    print('MEDIAN:')
    print(df2.median())
    print()
    print('MEAN:')
    print(df2.mean())
    print()
    print('SUM:')
    print(print(pd.DataFrame(df2.sum()).T))
    
    return pd.DataFrame(df2.median(),columns = [pair]).T,pd.DataFrame(df2.mean(),columns = [pair]).T,pd.DataFrame(df2.sum(),columns = [pair]).T
def test_future4(df,pair,column = 'support_indicator'):
    if 'JPY' in pair:
        mult = 100
    else:
        mult = 100 * 100
        
    print('TESTING DRAWDOWN')
    c_col = df.columns.get_loc('c')
    o_col = df.columns.get_loc('o')
    l_col = df.columns.get_loc('l')
    col = df.columns.get_loc(column)
    dv_col = df.columns.get_loc('delta_vector')
    
    lst = []
    arr = df.values
    for i in range(arr.shape[0] - 101):
        if arr[i,col] == 1 and arr[i,dv_col] > 0:
            c = arr[i,o_col]

            lst.append([mult * (arr[i ,c_col] - c),
                        mult * (c - arr[i + 1,l_col]),
                        mult * (c - arr[i + 2,l_col] ),
                        mult * (c - arr[i + 3,l_col] ),
                        mult * (c - arr[i + 4,l_col] ), 
                        mult * (c - arr[i + 5,l_col] ), 
                        mult * (c - arr[i + 6,l_col] )
                       ])
    df2 = pd.DataFrame(lst,columns = ['current_hour','next_hour','next_two_hours','next_three_hours','next_four_hours','next_five_hours','next_six_hours'])  
    print('MEDIAN:')
    print(df2.median())
    print()
    print('MEAN:')
    print(df2.mean())
    print()
    print('SUM:')
    print(print(pd.DataFrame(df2.sum()).T))
    
    return pd.DataFrame(df2.median(),columns = [pair]).T,pd.DataFrame(df2.mean(),columns = [pair]).T,pd.DataFrame(df2.sum(),columns = [pair]).T
    return pd.DataFrame(df2.median(),columns = [pair]).T,pd.DataFrame(df2.mean(),columns = [pair]).T,pd.DataFrame(df2.sum(),columns = [pair]).T

def get_support(df):
    c = df.columns.get_loc('c')
    o = df.columns.get_loc('o')
    l = df.columns.get_loc('l')
    df['support_indicator'] = 0
    new_col = df.columns.get_loc('support_indicator')
    d = df.columns.get_loc('delta')
    
    arr = df.values
    for i in range(arr.shape[0]):
        if arr[i,c] == arr[i,l] and arr[i,o] - arr[i,c] >= .001 and arr[i,d] > 0 and arr[i,c] <= arr[i - 150,c] - .008:
            arr[i + 1,new_col] = 1
    df = pd.DataFrame(arr,columns = df.columns)
    print('SHAPE',df[df['support_indicator'] == 1].shape)
    return df

pair = 'EUR_USD'
df = load_df(pair = pair,granularity ='H1',start = datetime(2016,1,1,0,0,0),end = datetime(2022,7,31,0,0,0))
df = get_support(df)
test = test_future(df,pair,column = 'support_indicator')
test = test_future2(df,pair,column = 'support_indicator')
test = test_future3(df,pair,column = 'support_indicator')
test = test_future4(df,pair,column = 'support_indicator')
    

PATH: /Users/aidanmcconnell/Documents/MARKET_ANALYTICS/forex/Macro/EUR_USD_H1/2016-01-01_2022-07-31.csv
(41152, 15)
SHAPE (47, 16)
MEDIAN:
next_hour           0.8
next_two_hours      3.9
next_three_hours    1.6
next_four_hours     8.0
next_five_hours     5.2
next_six_hours      6.7
dtype: float64

MEAN:
next_hour            1.812766
next_two_hours       6.221277
next_three_hours     6.412766
next_four_hours      8.385106
next_five_hours      6.502128
next_six_hours      10.108511
dtype: float64

SUM:
   next_hour  next_two_hours  next_three_hours  next_four_hours  \
0       85.2           292.4             301.4            394.1   

   next_five_hours  next_six_hours  
0            305.6           475.1  
None
MEDIAN:
current_hour        -7.20
next_hour          -10.85
next_two_hours      -5.90
next_three_hours    -8.40
next_four_hours     -6.45
next_five_hours     -7.00
next_six_hours      -6.90
dtype: float64

MEAN:
current_hour        -9.01250
next_hour          -12.89375
next_two_h

In [112]:
min(df['l'].iloc[i:i+5])

1.02547

In [114]:
l_col = df.columns.get_loc('l')
for i in range(df.shape[0]):
    if df['support_indicator'].iloc[i] == 1 and df['delta_vector'].iloc[i] > 0:
        c = df['o'].iloc[i]
        min_ = min(df.iloc[i:i+5,l_col])
        print(10000 * (c - min_))

0.0
0.0
5.9000000000009045
15.600000000000058
0.0
0.9999999999998899
9.999999999998899
0.0
0.0
4.20000000000087
37.199999999999456
0.20000000000131024
8.60000000000083
9.399999999999409
0.0
0.20000000000131024
12.600000000000389
7.900000000000684
16.20000000000177
1.100000000000545
7.299999999998974
1.80000000000069
9.399999999999409
4.20000000000087
8.099999999999774
10.699999999999044
0.40000000000040004
9.599999999998499
7.800000000000029
8.40000000000174


In [133]:
num_steps = 6
l_col = df.columns.get_loc('l')
h_col = df.columns.get_loc('h')
c_col = df.columns.get_loc('c')
print('WINS')
l_col = df.columns.get_loc('l')
h_col = df.columns.get_loc('h')
wins = []
losses = []
for i in range(df.shape[0]):
    if df['support_indicator'].iloc[i] == 1 and df['delta_vector'].iloc[i] > 0:
        wins.append(1)
        c = df['o'].iloc[i]
        min_ = min(df.iloc[i:i+num_steps,l_col])
        max_ = max(df.iloc[i:i+num_steps,h_col])
        last = df.iloc[i+num_steps - 2,c_col]
        last2 = df.iloc[i+num_steps - 1,c_col]
        last3 = df.iloc[i+num_steps ,c_col]
        val1 = round(10000 * (min_ - c),1)
        val2 = round(10000 * (max_ - c),1)
        val3 = round(10000 * (last - c),1)
        val4 = round(10000 * (last2 - c),1)
        val5 = round(10000 * (last3 - c),1)
        print(val1,val2,val3,val4,val5)
print('WIN COUNT:',len(wins))
print()
print('LOSSES')

for i in range(df.shape[0]):
    if df['support_indicator'].iloc[i] == 1 and df['delta_vector'].iloc[i] < 0:
        losses.append(1)
        c = df['o'].iloc[i]
        min_ = min(df.iloc[i:i+num_steps,l_col])
        max_ = max(df.iloc[i:i+num_steps,h_col])
        last = df.iloc[i+num_steps - 2,c_col]
        last2 = df.iloc[i+num_steps - 1,c_col]
        last3 = df.iloc[i+num_steps ,c_col]
        val1 = round(10000 * (min_ - c),1)
        val2 = round(10000 * (max_ - c),1)
        val3 = round(10000 * (last - c),1)
        val4 = round(10000 * (last2 - c),1)
        val5 = round(10000 * (last3 - c),1)
        print(val1,val2,val3,val4,val5)
print('Loss COUNT:',len(losses))

WINS
0.0 15.4 4.9 10.8 13.0
0.0 53.4 45.6 51.3 50.9
-5.9 11.4 -2.4 0.6 17.5
-15.6 14.0 1.0 -8.8 -10.6
0.0 32.9 24.4 25.3 31.5
-1.0 44.4 35.7 42.8 44.8
-10.1 43.2 15.7 -3.6 -3.4
0.0 55.0 40.8 43.8 43.6
0.0 63.1 45.5 60.9 78.1
-4.2 26.4 24.4 25.8 11.0
-45.0 9.8 -35.4 -33.0 -34.8
-0.2 41.0 22.4 20.4 25.8
-8.6 33.1 16.5 21.5 25.1
-12.8 12.1 -4.0 -9.8 0.4
0.0 72.9 35.6 50.8 68.2
-2.0 49.3 33.8 10.0 28.7
-12.6 29.2 9.4 13.7 -3.5
-7.9 61.5 41.8 42.3 44.2
-16.2 26.7 -5.2 -2.1 -8.1
-1.1 35.5 16.7 16.1 14.2
-7.6 12.3 -6.6 0.0 0.7
-2.7 9.5 0.3 2.8 3.4
-9.4 50.9 40.1 47.5 53.3
-11.2 70.9 62.2 -10.4 -15.6
-8.1 30.8 22.7 13.0 20.8
-10.7 19.7 1.6 2.2 1.3
-0.4 57.4 43.4 43.6 40.7
-9.6 26.2 11.8 2.2 36.0
-7.8 77.6 55.8 38.3 32.0
-8.4 115.7 94.4 95.5 97.2
WIN COUNT: 30

LOSSES
-7.3 36.2 26.4 28.7 39.4
-109.7 0.8 -93.2 -83.6 -34.0
-24.6 16.2 5.6 14.2 5.6
-11.0 4.3 -7.1 -3.2 -6.5
-50.6 6.2 -42.2 -44.7 -33.8
-30.3 0.0 -20.4 -17.6 -21.1
-10.4 1.4 -5.8 -4.2 -13.8
-88.0 18.2 -79.8 -87.6 -89.1
-43.0 6.2 -12.5 

In [145]:
def test_results(df):
    
    l_col = df.columns.get_loc('l')
    h_col = df.columns.get_loc('h')
    o_col = df.columns.get_loc('o')
    col = df.columns.get_loc('support_indicator')
    
    arr = df.values
    sl = 10
    tp = 40
    steps_ahead = 5
    
    final_pips = 0
    for i in range(arr.shape[0]):
        if arr[i,col] == 1:
            val = arr[i,o_col]
            low = 10000*(val - min(arr[i:i + steps_ahead,l_col]))
            high = 10000*(max(arr[i:i + steps_ahead,h_col]) - val)
            last = 10000*(arr[i + steps_ahead - 1,c_col] - val)
            
            if low + 1 >= sl:
                final_pips = final_pips - sl
            elif high >= tp:
                final_pips = final_pips + tp
            else:
                final_pips = final_pips + last
            final_pips = round(final_pips)
            print(final_pips)
    print(final_pips)
test_results(df)

26
31
21
11
51
49
39
29
19
9
-1
23
13
49
39
79
119
143
133
155
145
135
151
191
181
171
211
201
241
231
271
261
251
241
258
251
251
241
281
304
294
284
324
314
354
344
384
384


In [149]:
val = 10000
r = .01
for i in range(40):
    print(i,val)
    val += (350 * ((val * r) / 10))
    
    

0 10000
1 13500.0
2 18225.0
3 24603.75
4 33215.0625
5 44840.334375
6 60534.451406249995
7 81721.50939843748
8 110324.03768789061
9 148937.45087865234
10 201065.55868618065
11 271438.5042263439
12 366441.98070556426
13 494696.67395251174
14 667840.5098358908
15 901584.6882784526
16 1217139.329175911
17 1643138.0943874796
18 2218236.427423097
19 2994619.177021181
20 4042735.888978595
21 5457693.450121103
22 7367886.157663489
23 9946646.31284571
24 13427972.52234171
25 18127762.905161306
26 24472479.921967763
27 33037847.89465648
28 44601094.65778625
29 60211477.78801144
30 81285495.01381545
31 109735418.26865086
32 148142814.66267866
33 199992799.7946162
34 269990279.7227318
35 364486877.62568796
36 492057284.79467875
37 664277334.4728163
38 896774401.5383021
39 1210645442.0767078


In [148]:
(384 * 10) * 20 * 2

153600

In [131]:
lst = [10.8,
51.3,
25.3,
43.8,
60.9,
50.8]
sum(lst)

242.89999999999998

In [132]:
40 * 2

80

In [135]:
100 * 20 * 2

4000

In [136]:
(100 * 20 * 2) * 20

80000

In [129]:
6 / 46

0.13043478260869565

In [139]:
(120 / 46) * 40

104.34782608695653

In [120]:
l_col = df.columns.get_loc('l')
h_col = df.columns.get_loc('h')
for i in range(df.shape[0]):
    if df['support_indicator'].iloc[i] == 1 and df['delta_vector'].iloc[i] > 0:
        c = df['o'].iloc[i]
        min_ = min(df.iloc[i:i+num_steps,l_col])
        max_ = max(df.iloc[i:i+num_steps,h_col])
        val1 = round(10000 * (c - min_),1)
        val2 = round(10000 * (max_ - c),1)
        print(val1,val2)

0.0 12.1
0.0 48.9
5.9 11.4
15.6 14.0
0.0 28.7
1.0 39.9
10.0 43.2
0.0 55.0
0.0 49.7
4.2 24.7
37.2 9.8
0.2 29.5
8.6 33.1
9.4 12.1
0.0 40.6
0.2 49.3
12.6 19.0
7.9 61.5
16.2 26.7
1.1 35.5
7.3 12.3
1.8 9.5
9.4 44.8
4.2 64.3
8.1 30.8
10.7 19.7
0.4 57.4
9.6 26.2
7.8 77.6
8.4 115.7


In [92]:
(df[df['support_indicator'] == 1]['delta_vector'].mean()) * 10000

1.5785714285713657

In [93]:
wins = df[(df['support_indicator'] == 1) & (df['delta_vector'] > 0)]
wins = wins.sort_values(by = 'delta_vector',ascending = False)
wins.shape

(43, 16)

In [87]:
df[(df['support_indicator'] == 1) & (df['delta_vector'] > 0) & (df['lower_wick'] >= .0003)].shape

(19, 16)

In [86]:
(df[(df['support_indicator'] == 1) & (df['delta_vector'] > 0)]['lower_wick'].mean()) * 10000

3.263043478260959

In [88]:
(df[(df['support_indicator'] == 1) & (df['delta_vector'] > 0) & (df['lower_wick'] >= .0003)]['lower_wick'].mean()) * 10000

6.742105263158239

In [90]:
df[(df['support_indicator'] == 1) & (df['delta_vector'] > 0) & (df['lower_wick'] <= .0001)].shape

(15, 16)

In [89]:
(df[(df['support_indicator'] == 1) & (df['delta_vector'] > 0) & (df['lower_wick'] <= .0001)]['delta_vector'].mean()) * 10000

11.353333333333548

In [7]:

def viz_support(df,col,start,end,ema):
    print('SELL INTO SUPPORT!!!!!')
    def print_support_example(df,col,ind_list,ind = 30,delta = 240,ema = True):
        i = ind_list[ind]   
        x = i


        print('SUPPORT ROW:',x,' SUPPORT NEXT TOUCH:',i, ' SUPPORT CLOSE VAL:')

        fig = go.Figure(data=go.Candlestick(x=df.iloc[i - delta:i + delta,:]['time'],
                            open=df.iloc[x - delta:i + delta,:]['o'],
                            high=df.iloc[x - delta:i + delta,:]['h'],
                            low=df.iloc[x - delta:i + delta,:]['l'],
                            close=df.iloc[x - delta:i + delta,:]['c']))   


        fig.add_vrect(x0=df['time'].iloc[i - 1], x1=df['time'].iloc[i+1], 
                      annotation_text="trigger point" , annotation_position="top left",
                      fillcolor="green", opacity=0.25, line_width=0)

        fig.add_shape(type='line',
                        x0=df.iloc[x,:]['time'],
                        y0=df.iloc[x,:]['c'],
                        x1=df.iloc[i,:]['time'],
                        y1=df.iloc[x ,:]['c'],
                        line=dict(color='Red',),
                        xref='x',
                        yref='y'
        )

        try:
            plot(fig)
        except:
            from plotly.offline import plot
            plot(fig)
            
            
    temp_df = df[df[col] == 1]
    print('SHAPE:',temp_df.shape)
    ind_list = temp_df.index
    for i in range(start,end):
        print(i)

        #i = 3
        print_support_example(df,col,ind_list,ind = i,delta = 100,ema = ema)
        i += 1

        
viz_support(df = df,col = 'xau_morning',start = 1600,end = 1630,ema = True)

SELL INTO SUPPORT!!!!!
SHAPE: (1707, 15)
1600
SUPPORT ROW: 36654  SUPPORT NEXT TOUCH: 36654  SUPPORT CLOSE VAL:
1601
SUPPORT ROW: 36677  SUPPORT NEXT TOUCH: 36677  SUPPORT CLOSE VAL:
1602
SUPPORT ROW: 36700  SUPPORT NEXT TOUCH: 36700  SUPPORT CLOSE VAL:
1603
SUPPORT ROW: 36723  SUPPORT NEXT TOUCH: 36723  SUPPORT CLOSE VAL:
1604
SUPPORT ROW: 36746  SUPPORT NEXT TOUCH: 36746  SUPPORT CLOSE VAL:
1605
SUPPORT ROW: 36769  SUPPORT NEXT TOUCH: 36769  SUPPORT CLOSE VAL:
1606
SUPPORT ROW: 36792  SUPPORT NEXT TOUCH: 36792  SUPPORT CLOSE VAL:
1607
SUPPORT ROW: 36815  SUPPORT NEXT TOUCH: 36815  SUPPORT CLOSE VAL:
1608
SUPPORT ROW: 36838  SUPPORT NEXT TOUCH: 36838  SUPPORT CLOSE VAL:
1609
SUPPORT ROW: 36861  SUPPORT NEXT TOUCH: 36861  SUPPORT CLOSE VAL:
1610
SUPPORT ROW: 36884  SUPPORT NEXT TOUCH: 36884  SUPPORT CLOSE VAL:
1611
SUPPORT ROW: 36907  SUPPORT NEXT TOUCH: 36907  SUPPORT CLOSE VAL:
1612
SUPPORT ROW: 36930  SUPPORT NEXT TOUCH: 36930  SUPPORT CLOSE VAL:
1613
SUPPORT ROW: 36953  SUPPORT NEX